In [35]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import numpy as np

In [28]:
# read img files into a list
# data_dir = './test_data/Mask/'
# # os.listdir(data_dir)

# filenames = [name for name in os.listdir(data_dir)
#             if os.path.splitext(name)[-1] == '.jpg']

# img_arr = imageio.imread(os.path.join(data_dir, filenames[0]))
# img_arr.shape

# batch_size = 5 # number of imgs in a batch
# batch = torch.zeros(batch_size, 3, 256, 256)

# for i, filename in enumerate(filenames):
#     img_arr = imageio.imread(os.path.join(data_dir, filename))
#     imt_t = torch.from_numpy(img_arr)
#     img_t = img_t.permute(2, 0, 1)
#     img_t = img_t[:3]
    

In [47]:
# load data_set

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
     transforms.ToTensor()
#     transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  # 均值为0 方差为1 的正态分布
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./test_data/', transform=transform)



# for img, label in train_dataset:
#     if (label == 0):
#         print(img.shape)

# # train 每四个为一组
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# # 显示图片
# to_pil_image = transforms.ToPILImage()
# for image, label in train_dataloader:
#     img = to_pil_image(image[0])
#     img.show()

torch.Size([3, 256, 256, 17])

In [48]:
# calculate mean of imgs in each RGB channel
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
print(imgs.shape)

imgs.view(3, -1).mean(dim=1)  # mean of (number of imgs) in each channel

torch.Size([3, 256, 256, 17])


tensor([0.4793, 0.4304, 0.3820])

In [50]:
imgs.view(3, -1).std(dim=1) # std of (number of imgs) in each channel

tensor([0.2940, 0.2931, 0.2957])

In [56]:
# load dataset, normarlize it
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4793, 0.4304, 0.3820], std=[0.2940, 0.2931, 0.2957]) 
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./test_data/', transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [61]:
for img, label in train_dataset:
    print(img.shape)
    print(label)



torch.Size([3, 256, 256])
0
torch.Size([3, 256, 256])
0
torch.Size([3, 256, 256])
0
torch.Size([3, 256, 256])
0
torch.Size([3, 256, 256])
0
torch.Size([3, 256, 256])
0
torch.Size([3, 256, 256])
1
torch.Size([3, 256, 256])
1
torch.Size([3, 256, 256])
1
torch.Size([3, 256, 256])
1
torch.Size([3, 256, 256])
1
torch.Size([3, 256, 256])
2
torch.Size([3, 256, 256])
2
torch.Size([3, 256, 256])
2
torch.Size([3, 256, 256])
2
torch.Size([3, 256, 256])
2
torch.Size([3, 256, 256])
2
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([1, 3, 256, 256])
torch.Size([1])


In [62]:
for imgs, labels in train_dataloader:
    print(imgs.shape)
    print(labels.shape)

torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([4, 3, 256, 256])
torch.Size([4])
torch.Size([1, 3, 256, 256])
torch.Size([1])
